In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

paths={}
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        paths[filename]=os.path.join(dirname, filename)

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


In [18]:
movies=pd.read_csv(paths['movies.csv'])
ratings=pd.read_csv(paths['ratings.csv'])
print("done")

done


# Movies Dataset

In [19]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
# Print to confirm loading
print("Movies shape:", movies.shape)
print("Ratings shape:", ratings.shape)

Movies shape: (62423, 3)
Ratings shape: (25000095, 4)


# Ratings Dataset

In [21]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


# I.Collaborative filtering: Movie based

## Description
The model finds a movie's index and retrieves all user ratings for that movie. It then compares these ratings to other movies to identify similar ones and recommends them based on user preferences.

In [22]:
# merging the two datasets
df=pd.merge(ratings,movies,on='movieId')

print(df.shape)

df.head()

(25000095, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [23]:
# Take the first 3% of the data
num_rows = int(len(df) * 0.03)  # Calculate 3% of the total rows
df_short = df.iloc[:num_rows]  # Take the first 3% of the data
print("Sampled Data shape:", df_short.shape)

#df_short = df.sample(frac=0.002, random_state=42)  # 0.2% of data

total_nan = df_short.isna().sum().sum()
print(total_nan)
df_short.head()

Sampled Data shape: (750002, 6)
0


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [24]:
user_item_matrix=df_short.pivot_table(index='movieId',columns='userId',values='rating').fillna(0)
print("User-Item Matrix shape:", user_item_matrix.shape)

User-Item Matrix shape: (20111, 5108)


# KNN

In [25]:
model_knn=NearestNeighbors(metric='cosine', algorithm = 'auto', n_neighbors=5, n_jobs=-1)
model_knn.fit(user_item_matrix)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [26]:
def recommend_movies(movie_title, model = model_knn, user_item_matrix = user_item_matrix, number_recommendations = 5):
    movie_index = df_short[df_short['title'] == movie_title].index[0]
    print('movie_index:',movie_index)
    
    distances,indexes = model.kneighbors(user_item_matrix.iloc[movie_index].values.reshape(1, -1),number_recommendations, return_distance=True)
    print('indexes',indexes)
    
    recommended_movies=df_short.iloc[indexes.flatten()]
    return recommended_movies
recommend_movies('Toy Story (1995)')

movie_index: 70
indexes [[  70  531 3547  651  730]]


,userId,movieId,rating,timestamp,title,genres
70,2,1,3.5,1141415820,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
531,3,8865,4.0,1439473526,Sky Captain and the World of Tomorrow (2004),Action|Adventure|Sci-Fi
3547,19,91658,2.0,1446298585,"Girl with the Dragon Tattoo, The (2011)",Drama|Thriller
651,3,59501,3.5,1566090550,"Chronicles of Narnia: Prince Caspian, The (2008)",Adventure|Children|Fantasy
730,3,81229,4.0,1439474063,Red (2010),Action|Comedy


# II.Collaborative filtering: Genre of movie based

Recommand movies based on the genres without taking into considerations the ratings and other attributes

In [27]:
movies_2=movies.copy()
movies_2['genres'] = movies_2['genres'].fillna('').str.split('|')

# Create genre features
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies_2['genres'])

# Create the genre dataframe
genre_df = pd.DataFrame(genre_matrix,columns=mlb.classes_,index=movies_2.index)

# Combine movie information with genre encoding
movies_with_genres = pd.concat([movies, genre_df], axis=1)

movies_with_genres.head()
print(movies_with_genres.shape)

(62423, 23)


In [28]:
movies_with_genres_short=movies_with_genres.drop(columns=['genres','title','movieId'])
movies_with_genres_short.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#KNN Model

In [29]:
knn_model_genres = NearestNeighbors(n_neighbors=5, algorithm='auto')
knn_model_genres.fit(movies_with_genres_short)

NearestNeighbors()

In [30]:
def recommend_movies_genre(movie_title, model = knn_model_genres, user_item_matrix = user_item_matrix, number_recommendations = 5):
    movie_index = movies_with_genres[df_short['title'] == movie_title].index[0]
    print('movie_index:',movie_index)
    
    distances,indexes = model.kneighbors(user_item_matrix.iloc[movie_index].values.reshape(1, -1),number_recommendations, return_distance=True)
    print('indexes',indexes)
    
    recommended_movies_genres=movies_with_genres.iloc[indexes.flatten()]
    return recommended_movies_genres
recommend_movies('Toy Story (1995)')

movie_index: 70
indexes [[  70  531 3547  651  730]]


,userId,movieId,rating,timestamp,title,genres
70,2,1,3.5,1141415820,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
531,3,8865,4.0,1439473526,Sky Captain and the World of Tomorrow (2004),Action|Adventure|Sci-Fi
3547,19,91658,2.0,1446298585,"Girl with the Dragon Tattoo, The (2011)",Drama|Thriller
651,3,59501,3.5,1566090550,"Chronicles of Narnia: Prince Caspian, The (2008)",Adventure|Children|Fantasy
730,3,81229,4.0,1439474063,Red (2010),Action|Comedy


# III.Collaborative filtering: Genre and ratings based

In [45]:
combined_matrix = pd.concat([user_item_matrix, movies_with_genres_short], axis=1).fillna(0)
combined_matrix.columns = combined_matrix.columns.astype(str)
print(" shape user_item_matrix (ratings) :", user_item_matrix.shape)
print(" shape movies with genres :", movies_with_genres.shape)

print("Combined Matrix shape (ratings + genres):", combined_matrix.shape)
combined_matrix.head()

 shape user_item_matrix (ratings) : (20111, 5108)
 shape movies with genres : (62423, 23)
Combined Matrix shape (ratings + genres): (72041, 5128)


,1,2,3,4,5,6,7,8,9,10,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [46]:
#KNN
model_knn_combined = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=5, n_jobs=-1)
model_knn_combined.fit(combined_matrix)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [49]:
def recommend_movies_with_genres(movie_title, model=model_knn_combined, combined_matrix=combined_matrix, number_recommendations=5):
    try:
        movie_index = df_short[df_short['title'] == movie_title].index[0]
        
        movie_row = combined_matrix.iloc[[movie_index]]
        
        # Get recommendations using KNN with both ratings and genres
        distances, indexes = model.kneighbors(movie_row, 
                                              number_recommendations, return_distance=True)

        recommended_movies = df_short.iloc[indexes.flatten()]
        return recommended_movies[['title', 'genres', 'rating']]
    
    except IndexError:
        print(f"Movie '{movie_title}' not found in the dataset.")
        return None

# Example usage
recommend_movies_with_genres('Toy Story (1995)')

,title,genres,rating
70,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5
651,"Chronicles of Narnia: Prince Caspian, The (2008)",Adventure|Children|Fantasy,3.5
4767,Betrayed (1988),Drama|Thriller,2.5
806,World War Z (2013),Action|Drama|Horror|IMAX,3.5
749,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,4.0
